In [1]:
%reset

In [2]:
prompt_configuration = 'prompt_scheme_verbose.pkl'

dataset_sample = 'data/medium_ubuntu.pkl'

dataset_summary = 'data/summary_medium_ubuntu.pkl'

dataset_description = 'data/description_medium_ubuntu.pkl'

In [3]:
import pickle
from prompt_creation import infoPrompt
import numpy as np

with open(prompt_configuration, 'rb') as f:
    prompt = pickle.load(f)
    
with open(dataset_sample, 'rb') as f:
    sample = pickle.load(f)

with open(dataset_summary, 'rb') as f:
    summary =pickle.load(f)

with open(dataset_description, 'rb') as f:
    description = pickle.load(f)

from prompt_creation import PromptLlamaMPC

prompt_generator = PromptLlamaMPC(prompt)

In [4]:
def conversation_basic(posts, speakers, user_tag):
    history = ""
    
    i = 1

    for p, s in zip(posts, speakers):
        history += str(i) + " : " + user_tag[s] + " [WRITES] " + p + "\n"
        i = i + 1

    history = history[:-1]

    return history

In [5]:
def conversation_interaction(speakers, addrs, user_tag):
    history = ""

    last_diff_user = 16
    last_user = 16
    
    i = 1

    for s, a in zip(speakers, addrs):
        final_a = a

        if last_user != s:
            last_diff_user = last_user
        if a == 17:
            final_a = last_diff_user
        last_user = s

        if final_a == 16:
            final_a = -1

        history += str(i) + " : " + user_tag[s] + " to " + user_tag[final_a] + "\n"
        
        i = i + 1

    history = history[:-1]

    return history

In [6]:
basic_conv = list()
struct_conv = list()

for s in sample:
    basic_conv.append(conversation_basic(s.context, s.spk_agents, ["[ALEX]", "[BENNY]", "[CAM]", "[DANA]", "[OTHER]"]))
    struct_conv.append(conversation_interaction(s.spk_agents, s.adr_agents, ["[ALEX]", "[BENNY]", "[CAM]", "[DANA]", "[OTHER]"]))

In [7]:
print(basic_conv[8])

1 : [ALEX] [WRITES] i would just set up vnc if i were you : https : //www.howtoforge.com/how-to-install-vnc-server-on-ubuntu-14.04
2 : [CAM] [WRITES] if you already have x running , then all you 'd need would be to `` display=:0 ebook-convert blah ''
3 : [ALEX] [WRITES] you can either re-connect to the vnc section when you need to come back to it , or you can use tmux and have to option to re-connect to the command via ssh
4 : [ALEX] [WRITES] s/vnc section/vnc server/
5 : [BENNY] [WRITES] um how do you start transmission ?
6 : [CAM] [WRITES] transmission-gtk
7 : [ALEX] [WRITES] how to block sftp ?
8 : [DANA] [WRITES] i 've set `export display=:0`
9 : [CAM] [WRITES] note - that only works if you do indeed have x running
10 : [DANA] [WRITES] and still the same message : can not connect to x server :0
11 : [BENNY] [WRITES] ( transmission-gtk:3383 ) : gtk-warning ** : can not open display :
12 : [DANA] [WRITES] ok , maybe i 'll try with vnc
13 : [DANA] [WRITES] thanks !
14 : [CAM] [WRITES]

In [8]:
print(struct_conv[8])

1 : [ALEX] to [DANA]
2 : [CAM] to [ALEX]
3 : [ALEX] to [DANA]
4 : [ALEX] to [CAM]
5 : [BENNY] to [ALEX]
6 : [CAM] to [BENNY]
7 : [ALEX] to [OTHER]
8 : [DANA] to [CAM]
9 : [CAM] to [DANA]
10 : [DANA] to [CAM]
11 : [BENNY] to [DANA]
12 : [DANA] to [BENNY]
13 : [DANA] to [BENNY]
14 : [CAM] to [BENNY]
15 : [BENNY] to [CAM]


# SUMMARY

In [10]:
summary_prompts = []
user_tag = ["[ALEX]", "[BENNY]", "[CAM]", "[DANA]", "[OTHER]"]

for b_c, s_c in zip(basic_conv, struct_conv):

    item = dict()
    item['conversation'] = b_c
    item['interaction'] = s_c
    item['summary'] = ""
    item['description'] = ""

    summary_prompts.append(prompt_generator.summarization(item=item)[0])

print(summary_prompts[1])

with open('prompt/summary_prompts.pkl', 'wb') as f:
    pickle.dump(summary_prompts, f)

<s>[INST]<<SYS>>

You are a system working on conversations. 

A conversation is a sequence of messages exchanged among two or more users. 
Each message is a string of text.
Each message is associated with a speaker and an addressee.
Each message has an integer index according to the order of the messages in the conversation.
The speaker is the user who wrote the message.
The addressee is the user to whom the message is directed.
Each user can be the speaker of multiple messages and the addressee of multiple messages.

 You are given in input the following information about a conversation, listed in the following bullet point list.

 * The transcript of the messages of the conversation, reporting the index, the user id of the speaker and the text of each message in chronological order.

 * The transcript of the interactions of the conversation, reporting the index, the user id of the speaker and the user id of the addressee of each message in chronological order.

Based on the informat

# USER DESCRIPTION

In [11]:
description_prompts = []
user_tag = ["[ALEX]", "[BENNY]", "[CAM]", "[DANA]", "[OTHER]"]

for b_c, s_c in zip(basic_conv, struct_conv):

    item = dict()
    item['conversation'] = b_c
    item['interaction'] = s_c
    item['summary'] = ""
    item['description'] = ""

    description_prompts.append(prompt_generator.describe_next_speaker(item=item)[0])

print(description_prompts[1])

with open('prompt/description_prompts.pkl', 'wb') as f:
    pickle.dump(description_prompts, f)

<s>[INST]<<SYS>>

You are a system working on conversations. 

A conversation is a sequence of messages exchanged among two or more users. 
Each message is a string of text.
Each message is associated with a speaker and an addressee.
Each message has an integer index according to the order of the messages in the conversation.
The speaker is the user who wrote the message.
The addressee is the user to whom the message is directed.
Each user can be the speaker of multiple messages and the addressee of multiple messages.

 You are given in input the following information about a conversation, listed in the following bullet point list.

 * The transcript of the messages of the conversation, reporting the index, the user id of the speaker and the text of each message in chronological order.

 * The transcript of the interactions of the conversation, reporting the index, the user id of the speaker and the user id of the addressee of each message in chronological order.

Based on the informat

# RESPONSE_SELECTION

CONV ONLY

In [12]:
rs_conv_prompts = []
rs_conv_struct_prompts = []
rs_struct_summ_prompts = []
rs_struct_desc_prompts = []
rs_struct_summ_desc_prompts = []

user_tag = ["[ALEX]", "[BENNY]", "[CAM]", "[DANA]", "[OTHER]"]

for b_c, s_c, summ, desc in zip(basic_conv, struct_conv, summary, description):
    
    item = dict()
    item['conversation'] = b_c
    item['interaction'] = s_c
    item['summary'] = summ
    item['description'] = desc


    rs_conv_prompts.append(prompt_generator.response_selection(conversation=True, item=item)[0])

    rs_conv_struct_prompts.append(prompt_generator.response_selection(conversation=True, interaction=True, item=item)[0])
    rs_struct_summ_prompts.append(prompt_generator.response_selection(interaction=True, summary=True, item=item)[0])

    rs_struct_desc_prompts.append(prompt_generator.response_selection(interaction=True, description=True, item=item)[0])
    rs_struct_summ_desc_prompts.append(prompt_generator.response_selection(interaction=True, summary=True, description=True, item=item)[0])


with open('prompt/rs_conv_prompts.pkl', 'wb') as f:
    pickle.dump(rs_conv_prompts, f)
    
with open('prompt/rs_conv_struct_prompts.pkl', 'wb') as f:
    pickle.dump(rs_conv_struct_prompts, f)
    
with open('prompt/rs_struct_summ_prompts.pkl', 'wb') as f:
    pickle.dump(rs_struct_summ_prompts, f)

with open('prompt/rs_struct_desc_prompts.pkl', 'wb') as f:
    pickle.dump(rs_struct_desc_prompts, f)

with open('prompt/rs_struct_summ_desc_prompts.pkl', 'wb') as f:
    pickle.dump(rs_struct_summ_desc_prompts, f)

# ADDRESSEE RECOGNITION


CONV ONLY

In [9]:
ar_conv_prompts = []
ar_conv_struct_prompts = []
ar_struct_summ_prompts = []
ar_struct_desc_prompts = []
ar_struct_summ_desc_prompts = []

ar_struct = []

user_tag = ["[ALEX]", "[BENNY]", "[CAM]", "[DANA]", "[OTHER]"]

for b_c, s_c, summ, desc in zip(basic_conv, struct_conv, summary, description):

    item = dict()
    item['conversation'] = b_c
    item['interaction'] = s_c
    item['summary'] = summ
    item['description'] = desc

    ar_conv_prompts.append(prompt_generator.addressee_recognition(conversation=True, item=item)[0])
    ar_conv_struct_prompts.append(prompt_generator.addressee_recognition(conversation=True, interaction=True, item=item)[0])
    ar_struct_summ_prompts.append(prompt_generator.addressee_recognition(summary=True, interaction=True, item=item)[0])
    ar_struct_desc_prompts.append(prompt_generator.addressee_recognition(interaction=True, description=True, item=item)[0])
    ar_struct_summ_desc_prompts.append(prompt_generator.addressee_recognition(summary=True, interaction=True, description=True, item=item)[0])
    ar_struct.append(prompt_generator.addressee_recognition(interaction=True, item=item)[0])


with open('prompt/ar_conv_prompts.pkl', 'wb') as f:
    pickle.dump(ar_conv_prompts, f)

with open('prompt/ar_conv_struct_prompts.pkl', 'wb') as f:
    pickle.dump(ar_conv_struct_prompts, f)

with open('prompt/ar_struct_summ_prompts.pkl', 'wb') as f:
    pickle.dump(ar_struct_summ_prompts, f)

with open('prompt/ar_struct_desc_prompts.pkl', 'wb') as f:
    pickle.dump(ar_struct_desc_prompts, f)

with open('prompt/ar_struct_summ_desc_prompts.pkl', 'wb') as f:
    pickle.dump(ar_struct_summ_desc_prompts, f)

with open('prompt/ar_struct_prompts.pkl', 'wb') as f:
    pickle.dump(ar_struct, f)